In [ ]:
import json
import pandas as pd
import datetime
import numpy as np

In [ ]:
def open_json(filename):
    """
    This Function Converts Json Lines to Pandas DataFrame
    parameters: Location
    output    : Dataframe
    """
    with open(filename) as f:
        lines = f.read().splitlines()
    df = pd.DataFrame(lines)
    df.columns = ['json_element']
    df['json_element'].apply(json.loads)
    return pd.json_normalize(df['json_element'].apply(json.loads))

In [ ]:
### converting json files to dataframes
users = open_json('users.json')
receipts = open_json('receipts.json')
brands = open_json('brands.json')

In [ ]:
### renaming some columns for better readability and usability
users.rename(columns={'_id.$oid':'userId'},inplace=True)
brands.rename(columns={'_id.$oid':'brandId'},inplace=True)
receipts.rename(columns={'_id.$oid':'receiptId'},inplace=True)

In [ ]:
len(users)

In [ ]:
users['userId'].nunique()

I noticed there are 495 total rows in users table - out of this only 212 are unique. Now, that's interesting!
There might be duplicates. Let's check that.

In [ ]:
users.drop_duplicates(inplace=True)

In [ ]:
len(users)

After dropping duplicates - we are left with 212 rows. So, here our hypothesis was right.

Similarly, lets check the same for other two tables.

In [ ]:
len(brands) ==  brands['brandId'].nunique() #all rows are unique

In [ ]:
len(receipts) ==  receipts['receiptId'].nunique() #all rows are unique

Here, the *users* and *brands* tables are in good form. I say they are in 3NF and are ready to be the data warehouse.

Now, lets have a look at *receipts* table

Here, there are lots of *NaN* present - which is one of the Data Quality issues that we will discuss in the later sections.
Basically, here, if there *NaN* in the *rewardsReceiptItemList* - it is not useful for the further analysis.So, we will remove the rows that do not have any data in *rewardsReceiptItemList* column

In [ ]:
receipts = receipts[receipts['rewardsReceiptItemList'].notna()] 
receipts

Here, the *rewardsReceiptItemList* has multiple values. In order to have a single value for each item in the receipt - I will explode this column.

In [ ]:
R = receipts.explode('rewardsReceiptItemList',ignore_index=True)
R

In SQL, we can use *json_extract* or some other in built fucntion to query specific brands info for the receipts we care about.
But, for the simplicity, I am going to extract barcodes from the *rewardsReceiptItemList* column.

Essentially, we would like to keep the *rewardsReceiptItemList* column as removing it will cause as loss of information

In [ ]:
R['barcode'] = ""
for idx, row in R.iterrows():
    try:
        if row['rewardsReceiptItemList']['barcode']:
            R.loc[idx,'barcode'] = row['rewardsReceiptItemList']['barcode']
            #row['barcode'] = row['rewardsReceiptItemList']['barcode']
    except:
        #print(row['rewardsReceiptItemList'])
        R.loc[idx,'barcode'] = 'No Barcode'
        pass

In [ ]:
R

Here, we have all the tables ready to go the Database/Datawarehouse.